In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [3]:
columns = [
    'movie_id', 'title', 'release_date', 'movie_popularity', 
    'vote_average', 'vote_count', 'budget', 'revenue', 'ratings', 'genres',
    'studios', 'actor_popularity_mean', 'director_popularity_mean'
]

target = ['revenue']

In [4]:
# Load the data
file_path = Path('../Tables/ML_Test_pgs_checked.csv')
df = pd.read_csv(file_path, index_col=0)[:-2]
# , index_col=0
# df = df.loc[:, columns].copy()

# # Drop the null columns where all values are null
# df = df.dropna(axis='columns', how='all')

# # Drop the null rows
# df = df.dropna()

# Encoded Age Ratings
ratings_enc = {
    'G': 1,
    'PG': 2,
    'PG-13': 3,
    'R': 4,
    'NC-17': 5,
    'NR':6
}

df['ratings_enc'] = df['ratings'].apply(lambda x: ratings_enc[x])

df['release_month'] = pd.DatetimeIndex(df['release_date']).month
df.head()

,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,actor_popularity_mean,director_popularity_mean,studios,genres,ratings_enc,release_month
movie_id,,,,,,,,,,,,,,
671,Harry Potter and the Philosopher's Stone,2001-11-16,268.472,7.9,21429,125000000,976475550,PG,2.993965,2.566,"[""'WarnerBros.Pictures'"", ""'HeydayFilms'"", ""'O...","['Adventure', ' Fantasy']",2,11
557,Spider-Man,2002-05-01,480.954,7.2,14421,139000000,821708551,PG-13,2.387022,2.914,"[""'Other'"", ""'ColumbiaPictures'"", ""'SonyPictur...","['Fantasy', ' Action']",3,5
672,Harry Potter and the Chamber of Secrets,2002-11-13,246.027,7.7,17294,100000000,876688482,PG,2.696712,2.566,"[""'WarnerBros.Pictures'"", ""'HeydayFilms'"", ""'O...","['Adventure', ' Fantasy']",2,11
673,Harry Potter and the Prisoner of Azkaban,2004-05-31,225.882,8.0,17001,130000000,789804554,PG,3.523069,3.333,"[""'WarnerBros.Pictures'"", ""'Other'"", ""'HeydayF...","['Adventure', ' Fantasy']",2,5
674,Harry Potter and the Goblet of Fire,2005-11-16,244.428,7.8,16341,150000000,895921036,PG-13,3.234944,2.695,"[""'WarnerBros.Pictures'"", ""'HeydayFilms'"", ""'O...","['Adventure', ' Fantasy', ' Family']",3,11


In [5]:
def profitability_df(df):
    if (df['revenue'] >= df['budget']*2):
        return True
    else:
        return False

df['profitability'] = df.apply(profitability_df, axis = 1)
df

,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,actor_popularity_mean,director_popularity_mean,studios,genres,ratings_enc,release_month,profitability
movie_id,,,,,,,,,,,,,,,
671,Harry Potter and the Philosopher's Stone,2001-11-16,268.472,7.9,21429,125000000,976475550,PG,2.993965,2.566,"[""'WarnerBros.Pictures'"", ""'HeydayFilms'"", ""'O...","['Adventure', ' Fantasy']",2,11,True
557,Spider-Man,2002-05-01,480.954,7.2,14421,139000000,821708551,PG-13,2.387022,2.914,"[""'Other'"", ""'ColumbiaPictures'"", ""'SonyPictur...","['Fantasy', ' Action']",3,5,True
672,Harry Potter and the Chamber of Secrets,2002-11-13,246.027,7.7,17294,100000000,876688482,PG,2.696712,2.566,"[""'WarnerBros.Pictures'"", ""'HeydayFilms'"", ""'O...","['Adventure', ' Fantasy']",2,11,True
673,Harry Potter and the Prisoner of Azkaban,2004-05-31,225.882,8.0,17001,130000000,789804554,PG,3.523069,3.333,"[""'WarnerBros.Pictures'"", ""'Other'"", ""'HeydayF...","['Adventure', ' Fantasy']",2,5,True
674,Harry Potter and the Goblet of Fire,2005-11-16,244.428,7.8,16341,150000000,895921036,PG-13,3.234944,2.695,"[""'WarnerBros.Pictures'"", ""'HeydayFilms'"", ""'O...","['Adventure', ' Fantasy', ' Family']",3,11,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370172,No Time to Die,2021-09-29,3366.389,7.6,2075,242000000,734000000,PG-13,4.164382,4.167,"[""'Other'"", ""'Metro-Goldwyn-Mayer'"", ""'Univers...","['Adventure', ' Action', ' Thriller']",3,9,True
580489,Venom: Let There Be Carnage,2021-09-30,5797.863,7.0,2452,110000000,454000000,PG-13,4.363038,13.077,"[""'MarvelEntertainment'"", ""'PascalPictures'"", ...","['Science Fiction', ' Action', ' Adventure']",3,9,True
610253,Halloween Kills,2021-10-14,616.978,6.9,1251,20000000,127000000,R,3.464000,2.385,"[""'UniversalPictures'"", ""'Other'""]","['Horror', ' Thriller']",4,10,True


In [6]:
# for test in df['studios']:
#     for studio in eval(test):
#         (studio)
        
# studio_revised = []
# for test in df['studios']:
#     holder = []
#     for studio in eval(test):
#         word = studio
#         if word not in holder:
#             holder.append(word)
#     studio_revised.append(holder)
    
# df.drop(['studios'],axis=1,inplace=True)
# df['studios'] = studio_revised

# df

In [7]:
# creating instance of labelencoder
labelencoder = LabelEncoder()

df['studios_cat'] = labelencoder.fit_transform(df.studios.values)
df['genres_cat'] = labelencoder.fit_transform(df.genres.values)

In [8]:
df = df.drop(['release_date', 'title', 'ratings', 'revenue'], axis=1)
df = df.drop(['studios', 'genres'], axis=1)
df.head()

movie_popularity  vote_average  vote_count     budget  \
movie_id                                                          
671                268.472           7.9       21429  125000000   
557                480.954           7.2       14421  139000000   
672                246.027           7.7       17294  100000000   
673                225.882           8.0       17001  130000000   
674                244.428           7.8       16341  150000000   

          actor_popularity_mean  director_popularity_mean  ratings_enc  \
movie_id                                                                 
671                    2.993965                     2.566            2   
557                    2.387022                     2.914            3   
672                    2.696712                     2.566            2   
673                    3.523069                     3.333            2   
674                    3.234944                     2.695            3   

          release_month  profitability  studios_cat  genres_cat  
movie_id                                                         
671                  11           True           25          18  
557                   5           True           10          36  
672                  11           True           25          18  
673                   5           True           28          18  
674                  11           True           25          16

In [9]:
# Create our features
X = pd.get_dummies(df.drop(columns='profitability'))


# Create our target
y = pd.get_dummies(df['profitability'])

In [10]:
X.describe()

,movie_popularity,vote_average,vote_count,budget,actor_popularity_mean,director_popularity_mean,ratings_enc,release_month,studios_cat,genres_cat
count,72.000000,72.000000,72.000000,7.200000e+01,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,641.271958,7.427778,6849.861111,1.147528e+08,3.763272,3.496674,3.027778,6.972222,17.125000,21.319444
std,1006.994160,0.629604,6819.869858,8.483708e+07,1.819889,2.548206,0.768614,2.973935,7.319543,14.684308
min,177.734000,6.000000,71.000000,1.000000e+06,1.318528,0.600000,2.000000,1.000000,0.000000,0.000000
25%,233.733750,6.975000,1565.500000,3.975000e+07,2.680786,1.765250,3.000000,4.750000,12.750000,8.000000
50%,305.311500,7.600000,3982.500000,1.050000e+08,3.208295,2.818500,3.000000,7.000000,20.000000,18.500000
75%,624.764500,7.900000,10683.750000,1.850000e+08,4.285281,4.248250,3.000000,9.000000,22.000000,35.250000
max,5884.885000,8.400000,25738.000000,3.560000e+08,11.495923,13.329000,6.000000,12.000000,29.000000,47.000000


In [11]:
# Check the balance of our target values
y = df['profitability']
y.value_counts()

True     50
False    22
Name: profitability, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({False: 15, True: 39})

# Balanced Random Forest Classifier

In [13]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=50)
brfc.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=50)

In [14]:
# Calculated the balanced accuracy score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5

In [15]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 0,  7],
       [ 0, 11]], dtype=int64)

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       0.00      0.00      1.00      0.00      0.00      0.00         7
       True       0.61      1.00      0.00      0.76      0.00      0.00        11

avg / total       0.37      0.61      0.39      0.46      0.00      0.00        18



In [17]:
# List the features sorted in descending order by feature importance
sorted(zip(brfc.feature_importances_, X.columns), reverse=True)

[(0.2198984816912784, 'vote_count'),
 (0.11153669599446153, 'director_popularity_mean'),
 (0.10406991776939929, 'studios_cat'),
 (0.10212829512259708, 'budget'),
 (0.09396103079303375, 'movie_popularity'),
 (0.0927410781571207, 'vote_average'),
 (0.0878987809786926, 'actor_popularity_mean'),
 (0.07451584138756076, 'genres_cat'),
 (0.06503261998617242, 'release_month'),
 (0.048217258119683394, 'ratings_enc')]

# Easy Ensemble AdaBoost Classifier

In [18]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=300, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=300, random_state=1)

In [19]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8636363636363636

In [20]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[7, 0],
       [3, 8]], dtype=int64)

In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       0.70      1.00      0.73      0.82      0.85      0.75         7
       True       1.00      0.73      1.00      0.84      0.85      0.71        11

avg / total       0.88      0.83      0.89      0.83      0.85      0.72        18



# Extra ML

In [22]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [24]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

predictions

array([ True, False,  True,  True, False,  True, False, False,  True,
        True, False,  True,  True, False,  True, False,  True,  True])